In [8]:
from pathlib import Path
import gc

import pandas as pd

import utilities.utilities as util

pd.set_option('display.max_columns', 500)

### Verify `fastparquet` is installed
We will need `fastparquet` to store Pandas DataFrame with string categorial data. Note: `pyarrow` will not support this!
If the following line crashes, install `snappy` and `fastparquet` through conda by executing: `conda install -c conda-forge python-snappy fastparquet snappy`

In [2]:
pd.io.parquet.get_engine('fastparquet')  # if this line crash, run 'conda install -c conda-forge python-snappy fastparquet snappy'

In [3]:
pd.show_versions()  # Verify: 'fastparquet : 0.5.0'


INSTALLED VERSIONS
------------------
commit           : db08276bc116c438d3fdee492026f8223584c477
python           : 3.8.5.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
Version          : 10.0.19041
machine          : AMD64
processor        : Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : English_United States.1252

pandas           : 1.1.3
numpy            : 1.19.2
pytz             : 2020.1
dateutil         : 2.8.1
pip              : 20.2.4
setuptools       : 50.3.1.post20201107
Cython           : 0.29.21
pytest           : 6.1.1
hypothesis       : None
sphinx           : 3.2.1
blosc            : None
feather          : None
xlsxwriter       : 1.3.7
lxml.etree       : 4.6.1
html5lib         : 1.1
pymysql          : None
psycopg2         : None
jinja2           : 2.11.2
IPython          : 7.19.0
pandas_datareader: None
bs4              : 4.9.3
bottleneck  

In [4]:
# Force 'fastparquet'
pd.set_option("io.parquet.engine", 'fastparquet')  

# Creating the Metatable

In [6]:
# Change paths accordingly
logs_path = Path('D:') / 'logs'
output_path = Path('meta_tables')
output_path.mkdir(parents=True, exist_ok=True)

### Generate MetaTable for each Year
We will create a checkpoint for each year as a protection against running out of memory.

In [ ]:
years = util.get_all_logs_annually(logs_path)

for year, logs in years:

    rows = []

    for log_json in logs:

        log = json.load(log_json.open())

        previous_dealer = 0  # Retained state per round
        round_wind = 0
        seat_wind = 0  # From player 0's POV

        for round_number, actions in enumerate(log['rounds']):

            init = actions.pop(0)
            if init['tag'] != 'INIT':
                raise Exception(f"{log_json.name} does not have INIT!")

            # Check if dealership has been transferred
            current_dealer = int(init['data']['oya'])
            if previous_dealer != current_dealer:
                previous_dealer = current_dealer

                # Check if game has completed a full circle
                if current_dealer == 0:
                    round_wind += 1
                    if round_wind > 3:
                        round_wind = 0
                # round_wind, seat_wind = next_seat_wind(round_wind, seat_wind)

            # Check if there's any winners or exhaustive/abortive draw
            winner = -1
            if actions[-1]['tag'] == 'AGARI':
                winner = actions[-1]['data']['winner']

            # Scores
            scores = init['data']['scores']
            end_scores = actions[-1]['data']['scores']

            # Row Creation
            rows.append({
                'log_id': log_json.stem,
                'round': round_number,

                'round_wind': round_wind,
                'dealer': current_dealer,
                'winner': winner,

                # 'seat_wind': seat_wind,

                'honba': init['data']['combo'],
                'riichibo': init['data']['reach'],


                'p0_start_score': scores[0],
                'p1_start_score': scores[1],
                'p2_start_score': scores[2],
                'p3_start_score': scores[3],

                'p0_end_score': end_scores[0],
                'p1_end_score': end_scores[1],
                'p2_end_score': end_scores[2],
                'p3_end_score': end_scores[3],
            })

    df = pd.DataFrame(rows)
    df.to_parquet(output_path / f'{year}.parquet', engine='fastparquet')
    df = None
    gc.collect()  # Potentially get some memory back

## Merge all MetaTables into a single DataFrame

In [ ]:
dataframes = [pd.read_parquet(mt) for mt in output_path.iterdir()]
accumulated = pd.concat(dataframes)

In [9]:
accumulated.memory_usage(deep=True)

Index             420008141
round_wind        184902080
dealer            184902080
winner            184902080
honba             184902080
riichibo          184902080
p0_start_score    184902080
p1_start_score    184902080
p2_start_score    184902080
p3_start_score    184902080
p0_end_score      184902080
p1_end_score      184902080
p2_end_score      184902080
p3_end_score      184902080
dtype: int64

In [10]:
accumulated.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 23112760 entries, ('2009020103gm-00a9-0000-2453a04c', 0) to ('2019123123gm-00e1-0000-f7f33877', 4)
Data columns (total 13 columns):
 #   Column          Dtype
---  ------          -----
 0   round_wind      int64
 1   dealer          int64
 2   winner          int64
 3   honba           int64
 4   riichibo        int64
 5   p0_start_score  int64
 6   p1_start_score  int64
 7   p2_start_score  int64
 8   p3_start_score  int64
 9   p0_end_score    int64
 10  p1_end_score    int64
 11  p2_end_score    int64
 12  p3_end_score    int64
dtypes: int64(13)
memory usage: 2.4+ GB


#### Change Column Type to lower memory usage
We want to lower the memory usage by our Merged MetaTable DataFrame, from about 2 GB -> 1 GB.

In [11]:
accumulated.reset_index(inplace=True)

accumulated['log_id'] = accumulated['log_id'].astype('category')
accumulated['round'] = accumulated['round'].astype('category')

categorical_cols = ['round_wind', 'dealer', 'winner']
accumulated[categorical_cols] = accumulated[categorical_cols].astype('category')

cols = ['honba', 'riichibo']
for col in cols:
    accumulated[col] = pd.to_numeric(accumulated[col], downcast='unsigned')

for i in range(4):
    accumulated[f'p{i}_start_score'] = pd.to_numeric(accumulated[f'p{i}_start_score'], downcast='integer')
    accumulated[f'p{i}_end_score'] = pd.to_numeric(accumulated[f'p{i}_end_score'], downcast='integer')

accumulated.set_index(['log_id', 'round'], inplace=True)  # Create MultiIndex

In [12]:
accumulated.memory_usage(deep=True)

Index             430033475
round_wind         23112952
dealer             23112952
winner             23112960
honba              23112760
riichibo           23112760
p0_start_score     92451040
p1_start_score     92451040
p2_start_score     92451040
p3_start_score     92451040
p0_end_score       92451040
p1_end_score       92451040
p2_end_score       92451040
p3_end_score       92451040
dtype: int64

In [13]:
accumulated.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 23112760 entries, ('2009020103gm-00a9-0000-2453a04c', 0) to ('2019123123gm-00e1-0000-f7f33877', 4)
Data columns (total 13 columns):
 #   Column          Dtype   
---  ------          -----   
 0   round_wind      category
 1   dealer          category
 2   winner          category
 3   honba           uint8   
 4   riichibo        uint8   
 5   p0_start_score  int32   
 6   p1_start_score  int32   
 7   p2_start_score  int32   
 8   p3_start_score  int32   
 9   p0_end_score    int32   
 10  p1_end_score    int32   
 11  p2_end_score    int32   
 12  p3_end_score    int32   
dtypes: category(3), int32(8), uint8(2)
memory usage: 1.0 GB


In [14]:
accumulated.to_parquet(Path('E:') / 'mahjong' / 'pandas' / 'log_round_meta.parquet', engine='fastparquet')

### Test if newly created optimized version works

In [4]:
accumulated = pd.read_parquet(Path('E:') / 'mahjong' / 'pandas' / 'log_round_meta.parquet', engine='fastparquet')

In [5]:
accumulated.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 23112760 entries, ('2009020103gm-00a9-0000-2453a04c', 0) to ('2019123123gm-00e1-0000-f7f33877', 4)
Data columns (total 13 columns):
 #   Column          Dtype   
---  ------          -----   
 0   round_wind      category
 1   dealer          category
 2   winner          category
 3   honba           uint8   
 4   riichibo        uint8   
 5   p0_start_score  int32   
 6   p1_start_score  int32   
 7   p2_start_score  int32   
 8   p3_start_score  int32   
 9   p0_end_score    int32   
 10  p1_end_score    int32   
 11  p2_end_score    int32   
 12  p3_end_score    int32   
dtypes: category(3), int32(8), uint8(2)
memory usage: 1011.0+ MB


In [8]:
accumulated.loc['2019123123gm-00e1-0000-f7f33877', 3]  # Accessing MultiIndexed

round_wind            0
dealer                2
winner                3
honba                 0
riichibo              0
p0_start_score    16400
p1_start_score    45900
p2_start_score    13400
p3_start_score    24300
p0_end_score      16400
p1_end_score      45900
p2_end_score      13400
p3_end_score      24300
Name: (2019123123gm-00e1-0000-f7f33877, 3), dtype: int64

In [9]:
accumulated.shape

(23112760, 13)

In [10]:
accumulated

round_wind dealer winner  honba  \
log_id                          round                                   
2009020103gm-00a9-0000-2453a04c 0              0      0     -1      0   
                                1              0      1     -1      1   
                                2              0      2     -1      2   
                                3              0      3     -1      3   
                                4              1      0     -1      4   
...                                          ...    ...    ...    ...   
2019123123gm-00e1-0000-f7f33877 0              0      0      3      0   
                                1              0      1      1      0   
                                2              0      1      0      1   
                                3              0      2      3      0   
                                4              0      3      0      0   

                                       riichibo  p0_start_score  \
log_id                          round                             
2009020103gm-00a9-0000-2453a04c 0             0           25000   
                                1             0           25000   
                                2             0           25000   
                                3             0           25000   
                                4             0           25000   
...                                         ...             ...   
2019123123gm-00e1-0000-f7f33877 0             0           25000   
                                1             0           21100   
                                2             0           13100   
                                3             0           16400   
                                4             0           16400   

                                       p1_start_score  p2_start_score  \
log_id                          round                                   
2009020103gm-00a9-0000-2453a04c 0               25000           25000   
                                1               25000           25000   
                                2               25000           25000   
                                3               25000           25000   
                                4               25000           25000   
...                                               ...             ...   
2019123123gm-00e1-0000-f7f33877 0               25000           25000   
                                1               23000           23000   
                                2               47000           15000   
                                3               45900           13400   
                                4               45900           12100   

                                       p3_start_score  p0_end_score  \
log_id                          round                                 
2009020103gm-00a9-0000-2453a04c 0               25000         25000   
                                1               25000         25000   
                                2               25000         25000   
                                3               25000         25000   
                                4               25000         25000   
...                                               ...           ...   
2019123123gm-00e1-0000-f7f33877 0               25000         25000   
                                1               32900         21100   
                                2               24900         13100   
                                3               24300         16400   
                                4               25600         16400   

                                       p1_end_score  p2_end_score  \
log_id                          round                               
2009020103gm-00a9-0000-2453a04c 0             25000         25000   
                                1             25000         25000   
                                2             25000         25000 